# Car detecion from aerial ortophoto with YOLOv7

This notebook is just a code suplementary for `Deepness` QGIS plugin documentation.
Please visit the documentation to see how easily use the model in QGIS: https://qgis-plugin-deepness.readthedocs.io/


Model trained with dataset ITCVD ( https://arxiv.org/pdf/1801.07339.pdf )
Spatial resolution of model: 10 cm/pixel.

In this notebook we go though the training process, including data preprocessing and model export.

## Download yolov7
Download yolov7 repository, install requirements, and put this notebook in the root directory of the repository:

``` 
git clone https://github.com/WongKinYiu/yolov7 
pip install -r yolov7/requirements.txt
pip install scipy onnx
cp $THIS_NOTEBOOK ./yolov7 

```

In [ ]:
import os
import numpy as np
import scipy.io
from PIL import Image

## Dataset manual download

Download manually ITCVD dataset.

At the moment of writing available at https://doi.org/10.17026/dans-xnc-h2fu

Create subdirectory for our data:
```
mkdir -p car/itcvd
```

Put the content of downloaded data `ITCVD/ITC_VD_Training_Testing_set/Training` in `./car/itcvd` (So that we will have a directory `./car/itcvd/GT` and `./car/itcvd/Images`)

In [ ]:
BASE_INPUT_DATA_DIR = 'car/itcvd/'
BASE_OUTPUT_DATA_DIR = 'car/'

img_output_dir = os.path.join(BASE_OUTPUT_DATA_DIR, 'images', 'all')
label_output_dir = os.path.join(BASE_OUTPUT_DATA_DIR, 'labels', 'all')

## Data preprocessing and preparation

The dataset images and labels are not suitable for yolov7. The images need to be cut into smaller parts.

In [ ]:
subimg_number = 0

img_overlap = 0.05
target_img_size = 640  # both dimensions equal
stride = int(target_img_size * (1 - img_overlap))


os.makedirs(img_output_dir, exist_ok=True)
os.makedirs(label_output_dir, exist_ok=True)

input_img_dir = os.path.join(BASE_INPUT_DATA_DIR, 'Image/')
all_output_img_names = []

for file_name in sorted(os.listdir(input_img_dir)):
    if not file_name.endswith('.jpg'):
        continue

    input_image_number = file_name.strip('.jpeg')
    mat_file_name = input_image_number + '.mat'
    labels_mat_path = os.path.join(BASE_INPUT_DATA_DIR, f'GT/{mat_file_name}')
    img_path = os.path.join(input_img_dir, f'{file_name}')

    labels_mat = scipy.io.loadmat(labels_mat_path)
    full_img = Image.open(img_path)
    full_img_numpy = np.array(full_img)

    car_labels = labels_mat[f'x{input_image_number}']

    input_img_size = full_img_numpy.shape
    bins_x = (input_img_size[1] - target_img_size) // stride + 1
    bins_y = (input_img_size[0] - target_img_size) // stride + 1

    for i in range(bins_x):
        for j in range(bins_y):
            x_start = i * stride
            y_start = j * stride
            x_end = x_start + target_img_size
            y_end = y_start + target_img_size

            if subimg_number == 1:
                a = 1

            yolo_labels = []
            for car_label in car_labels:
                x_left, y_upper, x_right, y_bottom, _, _ = car_label
                x_centre, y_centre = (x_left + x_right) / 2, (y_upper + y_bottom) / 2
                if x_centre > x_start and x_centre < x_end and y_centre < y_end and y_centre > y_start:
                    x_relative_centre = (x_centre - x_start) / target_img_size
                    y_relative_centre = (y_centre - y_start) / target_img_size
                    x_relative_width = (x_right - x_left) / target_img_size
                    y_relative_width = (y_bottom - y_upper) / target_img_size
                    class_id = 0
                    # yolo_label = [class_id, x_relative_centre, y_relative_centre, x_relative_width, y_relative_width]
                    yolo_label = [class_id, x_relative_centre, y_relative_centre, x_relative_width, y_relative_width]
                    yolo_labels.append(list(map(str, yolo_label)))

            img_slice = full_img_numpy[y_start:y_end, x_start:x_end]
            
            file_base_name = 's' + str(subimg_number).zfill(5)
            img_file_path = os.path.join(img_output_dir, f'{file_base_name}.jpg')
            subimg_number += 1
            im = Image.fromarray(img_slice)
            im.save(img_file_path)
            all_output_img_names.append(f'{file_base_name}.jpg')

            label_file_path = os.path.join(label_output_dir, f'{file_base_name}.txt')
            txt = '\n'.join([' '.join(label) for label in yolo_labels])
            with open(label_file_path, 'wt') as file:
                file.write(txt)            

### Create configuration files for yolo, with list of training/testing files

In [ ]:
images_numbers = list(range(subimg_number))

x = np.array(all_output_img_names)
N = len(x)
np.random.seed(44)
indices = np.random.permutation(N)
training_idx, test_idx, val_idx = indices[:int(0.8*N)], indices[int(0.8*N):int(0.9*N)], indices[int(0.9*N):]
training, test, val = x[training_idx], x[test_idx], x[val_idx]


def create_as_images_listing(set_name, file_numbers):
    txts = []
    for file_number in file_numbers:
        img_file_name = f'{file_number}'
        txts.append(f'./images/all/{img_file_name}') 

    txt = '\n'.join(txts)
    txt_file_path = os.path.join(BASE_OUTPUT_DATA_DIR, set_name + '.txt')
    with open(txt_file_path, 'wt') as file:
        file.write(txt) 
        

create_as_images_listing('train', sorted(training))
create_as_images_listing('test', sorted(test))
create_as_images_listing('val', sorted(val))

In [ ]:
CAR_YAML_CONENT = """
# CAR

# download command/URL (optional)
download: echo "no download"

# train and val data as 1) directory: path/images/, 2) file: path/images.txt, or 3) list: [path1/images/, path2/images/]
train: ./car/train.txt
val: ./car/val.txt
test: ./car/test.txt

# number of classes
nc: 1

# class names
names: ["car"]
"""

with open('./car/car.yaml', 'wt') as file:
    file.write(CAR_YAML_CONENT) 

# Run the training

In [ ]:
!python train.py \
    --workers 8 \
    --device 0 \
    --batch-size 2 \
    --data car/car.yaml \
    --img 640 640 \
    --cfg cfg/training/yolov7-tiny.yaml \
    --weights yolov7-tiny.pt \
    --name yolov7-car-detector \
    --hyp data/hyp.scratch.custom.yaml \
    --epochs 15 

## Run the testing

In [ ]:
!python test.py  \
    --device 0 \
    --batch-size 2 \ 
    --data car/car.yaml \
    --img 640  \
    --weights runs/train/yolov7-car-detector/weights/best.pt_xx \
    --name yolov7-car-detector

## Export the model to ONNX

In [ ]:
!python export.py \
    --weights runs/train/yolov7-car-detector/weights/best.pt \
    --grid \
    --simplify \
    --img-size 640 640

# Add metadata for `Deepness` plugin to run the model smoothly

In [ ]:
import json
import onnx

model = onnx.load('runs/train/yolov7-car-detector3/weights/best.onnx')

class_names = {
    0: 'car',
}

m1 = model.metadata_props.add()
m1.key = 'model_type'
m1.value = json.dumps('Detector')

m2 = model.metadata_props.add()
m2.key = 'class_names'
m2.value = json.dumps(class_names)

m3 = model.metadata_props.add()
m3.key = 'resolution'
m3.value = json.dumps(10)

m4 = model.metadata_props.add()
m4.key = 'tiles_overlap'
m4.value = json.dumps(10)

m4 = model.metadata_props.add()
m4.key = 'det_conf'
m4.value = json.dumps(0.3)

m4 = model.metadata_props.add()
m4.key = 'det_iou_thresh'
m4.value = json.dumps(0.7)


FIANL_MODEL_FILE_PATH = os.path.abspath('runs/train/yolov7-car-detector2/weights/car_aerial_detection_yolo7_ITCVD_deepness.onnx')
onnx.save(model, FIANL_MODEL_FILE_PATH)

print(f'Your ONNX model with metadata is at: {FIANL_MODEL_FILE_PATH}')

### Now you can load the model in Deepness!